<a href="https://colab.research.google.com/github/Blacksujit/Deep-Learning-Specialization-Repo/blob/main/DoctorChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio faiss-cpu PyPDF2 openai==0.28 tiktoken -U langchain-community

In [ ]:
import gradio as gr
import openai
import faiss
import numpy as np
import requests
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
import json


In [ ]:
# OpenAI ve diğer API anahtarlarını ayarlayın
openai_api_key = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxx"  # OpenAI API anahtarınızı buraya ekleyin
openai.api_key = openai_api_key

weather_api_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxx"  # OpenWeatherMap API anahtarınızı buraya ekleyin
exchange_api_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxx"  # Exchangeratesapi.io API anahtarınızı buraya ekleyin


In [ ]:
# PDF dosyalarının yolları
pdf_paths = ['/content/Current Essentials of Medicine.pdf'
]

# FAISS indeksi ve belgeler için global değişkenler
vector_index = None
documents = []
chat_history = []

In [ ]:
# PDF'leri okuma ve indeksleme fonksiyonu
def index_pdfs():
    global vector_index, documents

    for pdf_path in pdf_paths:
        pdf_reader = PdfReader(pdf_path)
        text = ""
        for page in pdf_reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
        documents.append(text)

    combined_text = " ".join(documents)

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_text(combined_text)

    embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
    vectors = embeddings.embed_documents(texts)

    vector_array = np.array(vectors)

    index = faiss.IndexFlatL2(vector_array.shape[1])
    index.add(vector_array)

    vector_index = index

    print("Bilgi tabanı başarıyla oluşturuldu!")

index_pdfs()

<ipython-input-8-b3abdacc56b4>:19: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)


Bilgi tabanı başarıyla oluşturuldu!


In [ ]:
# Hava durumu verilerini çekmek için fonksiyon
def fetch_weather(location):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={weather_api_key}&units=metric&lang=tr"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        temp = data['main']['temp']
        weather_description = data['weather'][0]['description']
        return f"{location} için anlık hava durumu: {temp}°C ve {weather_description}."
    else:
        return "Hava durumu bilgilerini alamadım. Lütfen konumu kontrol edip tekrar deneyin."


In [ ]:
# Döviz kuru verilerini çekmek ve döviz dönüşümü yapmak için fonksiyon
def fetch_exchange_rate(base_currency, target_currency, amount=1):
    url = f"https://api.exchangeratesapi.io/v1/latest?access_key={exchange_api_key}&format=1"
    response = requests.get(url)
    data = response.json()
    rates = data.get('rates', {})
    if target_currency in rates:
        rate = rates[target_currency]
        converted_amount = float(amount) * rate
        return f"{amount} {base_currency} = {converted_amount:.2f} {target_currency}."
    else:
        return f"{target_currency} için döviz kuru bilgisi bulunamadı."


In [ ]:
def format_chat_history(chat_history):
    formatted_history = ""
    for entry in chat_history:
        if entry["role"] == "user":
            formatted_history += f"<div class='chat-bubble user'>{entry['content']}</div>"
        else:
            formatted_history += f"<div class='chat-bubble assistant'>{entry['content']}</div>"

    return formatted_history

In [ ]:
# GPT-4 Yanıtını oluşturmak için fonksiyon (function calling ile)
def generate_gpt4_response(prompt_input):
    global vector_index, documents, chat_history
    openai.api_key = openai_api_key

    functions = [
        {
            "name": "fetch_weather",
            "description": "Belirli bir konum için hava durumu bilgisini alır.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "Hava durumu almak istediğiniz konumun adı"
                    }
                },
                "required": ["location"],
            },
        },
        {
            "name": "fetch_exchange_rate",
            "description": "Belirli iki para birimi arasındaki döviz kuru bilgisini alır ve isteğe bağlı olarak belirli bir miktar için dönüştürme yapar.",
            "parameters": {
                "type": "object",
                "properties": {
                    "base_currency": {
                        "type": "string",
                        "description": "Döviz kurunu almak istediğiniz temel para birimi, varsayılan olarak EUR'dir."
                    },
                    "target_currency": {
                        "type": "string",
                        "description": "Döviz kurunu almak istediğiniz hedef para biriminin ISO 4217 kodu (örneğin: TRY)"
                    },
                    "amount": {
                        "type": "number",
                        "description": "Dönüştürmek istediğiniz miktar (varsayılan olarak 1)."
                    }
                },
                "required": ["base_currency", "target_currency", "amount"],
            },
        }
    ]

    # OpenAI API çağrısı (function calling ile)
    response = openai.ChatCompletion.create(
        model='gpt-4o-mini',
        messages=[
            {"role": "system",
             "content": "Sen tıp bilgileri ile donatılmış bir asistansın ve görevin tıbbi konulardaki sorulara cevap vermektir."},
            {"role": "user", "content": prompt_input}
        ],
        functions=functions,
        function_call="auto",  # Modelin fonksiyon çağrısına karar vermesine izin ver
        temperature=0.5,
        max_tokens=512
    )

    # Sohbet geçmişini güncelle
    chat_history.append({"role": "user", "content": prompt_input})

    # Bir fonksiyon çağrısı istenip istenmediğini kontrol et
    if 'choices' in response and response['choices'][0]['finish_reason'] == 'function_call':
        function_call_info = response['choices'][0]['message']['function_call']
        function_name = function_call_info['name']
        arguments = json.loads(function_call_info['arguments'])

        if function_name == 'fetch_weather':
            location = arguments['location']
            weather_response = fetch_weather(location)
            chat_history.append({"role": "assistant", "content": weather_response})
            return format_chat_history(chat_history)
        elif function_name == 'fetch_exchange_rate':
            base_currency = arguments['base_currency']
            target_currency = arguments['target_currency']
            amount = arguments.get('amount', 1)
            exchange_rate_response = fetch_exchange_rate(base_currency, target_currency, amount)
            chat_history.append({"role": "assistant", "content": exchange_rate_response})
            return format_chat_history(chat_history)

    # Asistanın yanıtını al
    assistant_response = response['choices'][0]['message']['content'].strip()
    chat_history.append({"role": "assistant", "content": assistant_response})

    return format_chat_history(chat_history)

In [ ]:
with gr.Blocks() as demo:
    # CSS ile kaydırma özelliği ve stil düzenlemeleri
    custom_css = """
    /* Chat balonları için stil */
    .chat-bubble {
        padding: 10px;
        border-radius: 10px;
        margin-bottom: 10px;
        max-width: 60%;
        word-wrap: break-word;
    }

    .user {
        background-color: #d1e7dd;
        text-align: right;
        margin-left: auto;
    }

    .assistant {
        background-color: #f8d7da;
        text-align: left;
        margin-right: auto;
    }

    /* Kaydırılabilir sohbet kutusu */
    #output-box {
        height: 400px;  /* Sabit yükseklik */
        width: 100%;
        overflow-y: scroll !important;  /* Kaydırmayı gizle ama JS ile açacağız */
        padding: 10px;
        border: 1px solid #ccc;
        border-radius: 10px;
        background-color: #f8f9fa;
        margin-bottom: 20px;
    }

    /* Giriş kutusu */
    #input-box {
        height: 150px;
        width: 100%;
    }
    """

    output_textbox = gr.HTML(label="Yanıt", elem_id="output-box")  # HTML bileşeni ile sohbeti gösteriyoruz
    input_textbox = gr.Textbox(label="Sorunuzu girin", lines=4, elem_id="input-box")  # Giriş alanı

    # Sorgu gönderildiğinde çalıştırılacak fonksiyon
    def on_submit(prompt_input):
        response = generate_gpt4_response(prompt_input)
        return response, ""  # Giriş kutusunu temizle

    # Sohbeti temizlemek için fonksiyon
    def clear_chat():
        global chat_history
        chat_history.clear()
        return "", ""  # Hem giriş kutusunu hem de sohbeti temizle

    # Giriş kutusuna "Enter" basıldığında çalıştırılan fonksiyon
    input_textbox.submit(on_submit, inputs=input_textbox, outputs=[output_textbox, input_textbox])

    # Mesaj gönderme ve sohbeti temizleme butonları
    submit_btn = gr.Button("Gönder")
    clear_btn = gr.Button("Chat'i Temizle")

    submit_btn.click(on_submit, inputs=input_textbox, outputs=[output_textbox, input_textbox])
    clear_btn.click(clear_chat, outputs=[output_textbox, input_textbox])

    demo.css = custom_css
    demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://1046f76164fb165e08.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://b162ad70725dccd76c.gradio.live
Killing tunnel 127.0.0.1:7861 <> https://2eae138fb007aae19d.gradio.live
Killing tunnel 127.0.0.1:7862 <> https://1dcffbd5ed4db3ce8d.gradio.live
Killing tunnel 127.0.0.1:7863 <> https://c9b054571b98902445.gradio.live
Killing tunnel 127.0.0.1:7864 <> https://6311d0f05878886aba.gradio.live
Killing tunnel 127.0.0.1:7865 <> https://44bc8391d8b84b0e1d.gradio.live
Killing tunnel 127.0.0.1:7866 <> https://1046f76164fb165e08.gradio.live


In [6]:
# import pandas as pd
# import numpy as np
# # from sklearn.model_selection import LinearRegression

In [7]:
# New Approach Starts

In [9]:
# AI Doctor Bot - Colab Implementation
# Step-by-step implementation that can be later deployed to Hugging Face Spaces

# Install required packages
!pip install -qU gradio==4.24.0
!pip install -qU langchain==0.1.0 langchain-community==0.0.34
!pip install -qU transformers==4.41.2 sentence-transformers==2.7.0
!pip install -qU bitsandbytes==0.43.1 accelerate==0.29.3 faiss-cpu==1.8.0
!pip install -qU huggingface_hub langchain_community
# Import libraries
import os
import gradio as gr
from langchain_community.llms import HuggingFacePipeline
from langchain.prompts import ChatPromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import torch
from google.colab import files

# Authenticate with Hugging Face
from huggingface_hub import notebook_login
notebook_login()

# Configuration
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"

# Initialize components
def initialize_components():
    print("Initializing LLM...")

    # Configure 4-bit quantization
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )

    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        device_map="auto",
        torch_dtype=torch.float16
    )

    # Create text generation pipeline
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        max_new_tokens=512,
        temperature=0.3,
        top_p=0.95,
        repetition_penalty=1.15,
        do_sample=True,
    )

    llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

    # Load knowledge base
    print("\nPlease upload your medical knowledge text file:")
    uploaded = files.upload()
    kb_filename = next(iter(uploaded))

    loader = TextLoader(kb_filename)
    docs = loader.load()

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
        is_separator_regex=False,
    )

    split_docs = text_splitter.split_documents(docs)

    # Create embeddings and vector store
    print("Creating vector store...")
    embeddings = HuggingFaceEmbeddings(
        model_name=EMBEDDING_MODEL,
        model_kwargs={"device": "cpu"},
        encode_kwargs={"normalize_embeddings": True}
    )

    vectorstore = FAISS.from_documents(split_docs, embeddings)

    return llm, vectorstore

# Initialize components
llm, vectorstore = initialize_components()

# Create prompt template
prompt_template = """[INST] <<SYS>>
You are Dr. AI, a helpful and knowledgeable medical assistant. Follow these rules:
1. Provide accurate information based on the context
2. Be concise and clear
3. If unsure, say you don't know
4. Always recommend consulting a real doctor
5. For emergencies, provide clear instructions
<</SYS>>

Context: {context}

Question: {question}

Provide a helpful response following all instructions above. [/INST]"""

prompt = ChatPromptTemplate.from_template(prompt_template)

# Create QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

# Emergency detection keywords
EMERGENCY_KEYWORDS = [
    "chest pain", "heart attack", "stroke", "can't breathe",
    "difficulty breathing", "unconscious", "severe bleeding",
    "choking", "suicidal", "homicidal", "severe pain",
    "broken bone", "burn", "seizure", "allergic reaction"
]

# Response generation
def generate_response(message, history):
    # Convert to lowercase for case-insensitive matching
    lower_msg = message.lower()

    # Emergency detection
    if any(keyword in lower_msg for keyword in EMERGENCY_KEYWORDS):
        emergency_response = """🚨 **EMERGENCY ALERT** 🚨

Based on your description, this may be a medical emergency.

**Immediate Action Required:**
1. Call your local emergency number:
   - 🇺🇸 US: 911
   - 🇪🇺 EU: 112
   - 🇬🇧 UK: 999
   - 🇮🇳 India: 112 or 108
2. Stay calm and follow their instructions
3. Do not hang up until told to do so

⚠️ **Do not rely on this AI for emergency situations**"""
        return emergency_response

    try:
        # Get response from QA system
        result = qa_chain.invoke({"query": message})
        response = result["result"]

        # Post-process response
        if not response.strip():
            response = "I couldn't generate a response to your question."

        # Add disclaimer
        response += """

---
⚠️ **Important Disclaimer:**
This information is not medical advice. Always consult a qualified healthcare professional for diagnosis and treatment."""

        return response

    except Exception as e:
        return f"⚠️ An error occurred: {str(e)}\n\nPlease try again or consult a doctor directly."

# Gradio interface for Colab
def create_colab_interface():
    with gr.Blocks(theme=gr.themes.Soft()) as app:
        gr.Markdown("""
        # 🏥 AI Medical Assistant (Mistral-7B)
        **Important:** This is not a substitute for professional medical advice.
        """)

        chatbot = gr.Chatbot(height=500)
        with gr.Row():
            msg = gr.Textbox(label="Your health question", placeholder="Describe your symptoms...", scale=7)
            submit = gr.Button("Submit", variant="primary")

        clear = gr.ClearButton([msg, chatbot])

        msg.submit(generate_response, [msg, chatbot], [chatbot])
        submit.click(generate_response, [msg, chatbot], [chatbot])

        gr.Markdown("""
        ### 🔍 How to use:
        1. Describe your symptoms or ask a health question
        2. Receive general information (not medical advice)
        3. Always consult a real doctor for personal advice

        ### ⚠️ For emergencies:
        - This bot cannot handle emergencies
        - Call your local emergency number immediately
        """)

    return app

# Create and launch interface
print("Launching Colab interface...")
app = create_colab_interface()
app.launch(share=True, debug=False)

ERROR: Cannot install langchain-community==0.0.34 and langchain==0.1.0 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00


RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
empty_like method already has a different docstring